In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from multiprocessing import Pool
from functools import partial
from copy import deepcopy
import time, os, sys

from sklearn import linear_model, metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

import pickle
from scipy import stats

import mkl
mkl.set_num_threads(1)
nCPU= 30
N=5

In [2]:
file = '/media/eys/xwj/proteome/data/20210926_dict_matrix_20dataset.pkl'
with open(file, 'rb') as f:
    [ dict_dataset, df_summary]=pickle.load(f)

In [3]:
# data
datalist = list(dict_dataset.keys())

# 
model_dict = {
    "LR":  linear_model.LinearRegression(),
    "Lasso": linear_model.Lasso(alpha=0.02, max_iter=1e5),
    "HR": linear_model.HuberRegressor(),  #Linear regression model that is robust to outliers.
    "Ridge": linear_model.Ridge(),
#     "Bay": linear_model.BayesianRidge(),
    "SVR": SVR( gamma='scale'),
    "RFR": RandomForestRegressor(n_estimators=100, max_depth=3)
    }
method_feature_select = ["random","cosine","self"]
# feature numbers
list_topn = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 200, 1000, 5000]

#############
def create_res(list_topn):
    res = {}
    for mfs in method_feature_select:
        res[mfs] = {} 
        for topn in list_topn:
            # summary table
            res[mfs][topn] = {}
            res[mfs][topn]["summary"] = pd.DataFrame(index = model_dict , columns= 
                        ["mae_val",  "rmse_val",  "r2_val",   "mae_val_mean",  "rmse_val_mean", "r2_val_mean", 
                         "mae_test", "rmse_test","r2_test", "mae_test_mean", "rmse_test_mean","r2_test_mean"])
            res[mfs][topn]["time"] = pd.DataFrame(data = 0, dtype = int, index = model_dict, columns= ["val","test"])
            # detail data for all models
            for m in model_dict:
                res[mfs][topn][m]={}
                res[mfs][topn][m]["y_pred_val"], res[mfs][topn][m]["y_pred_test"] ={}, {}
                res[mfs][topn][m]["df_val_mae"], res[mfs][topn][m]["df_val_rmse"], res[mfs][topn][m]["df_val_r2"], \
                res[mfs][topn][m]["df_test_mae"], res[mfs][topn][m]["df_test_rmse"], res[mfs][topn][m]["df_test_r2"]  \
                    = df_template.copy(), df_template.copy(), df_template.copy(), df_template.copy(), df_template.copy(), df_template.copy() 
    return res

 
# comp_y_pred not comput gene error; only save y_predict value，later compute generic +gene specific 1:5 
def comp_y_pred(p):
    # usage: comp_y_pred("ENSG00000000419") 
    if mfs == "cosine":
        '''# use cosine similarity way to select topn , only use TRAIN set :  a better feature selection methods??'''
        select=X_train.columns[cos_top[p]]

    elif mfs == "random":
        '''# random extract features'''             
        idx = np.random.randint(X_train.shape[1], size=topn)
        select = X_train.columns[idx]

    else:
        pass
#     print(p, end=" ")
    X_val_select, X_train_select, y_val, y_train = X_val[select], X_train[select], Y_val[p], Y_train[p]
    #'''fit model with selected features'''
    my_model=model_dict[m].fit(X_train_select, y_train)
    y_predict = my_model.predict(X_val_select)
#     print(p)
    
    return y_predict


In [5]:
datalist

['colon_86_2014_labelfree',
 'prostate_65_2019_labelfree',
 'lung_76_2020_labelfree',
 'liver_62_2019_labelfree',
 'brain_71_2017_labelfree',
 'liver_318_2019_tmt',
 'uterus_115_2020_tmt',
 'lung_211_2020_tmt',
 'lung_89_2020_tmt',
 'colon_95_2019_tmt',
 'renal_185_2019_tmt',
 'brain_108_2021_tmt',
 'pedbrain_188_2020_tmt',
 'headneck_151_2021_tmt',
 'pancrea_140_2021_tmt',
 'lung_202_2021_tmt',
 'breast_122_2020_tmt',
 'breast_77_2016_itraq',
 'ovary_119_2016_itraq',
 'stomach_80_2019_itraq']

In [18]:
%%time
# df_summary = df_summary.reindex(list(dict_dataset.keys()))

run_cross_validation = True
run_test = True

list_model = list(model_dict.keys())

for mykey in ["colon_86_2014_labelfree"]:
# for mykey in datalist[13:14]: 
    # data transform到mean=0, std=1
    X = dict_dataset[mykey]["RNA"].transform(lambda x: (x-x.mean())/x.std(), axis=1).round(3).transpose()
    Y = dict_dataset[mykey]["protein"].transform(lambda x: (x-x.mean())/x.std(), axis=1).round(3).transpose()

    X_use, X_test, Y_use, Y_test = train_test_split(X, Y, test_size=0.2, random_state=123, shuffle=True)
    print(mykey, X.shape, Y.shape, X_use.shape, X_test.shape,  Y_use.shape, Y_test.shape)
    my_list_topn = list_topn + [X.shape[1]]
    print(my_list_topn)
    
    df_template = pd.DataFrame(index= Y_use.columns, columns= range(N))
    res = create_res(my_list_topn)
    
    ############## 1. rank all features by a method, run once for the data set, use the maxtrix later
    # split i have its own feature rankings
    if run_cross_validation == True:
        print("prepare training set.")
        # cross-validation on train set; 
        kf = KFold(n_splits=N, random_state=1, shuffle=True)
        dict_cos_rank_val = dict.fromkeys(range(N)) 

        i = 0
        for train_index, test_index in kf.split(Y_use): 
            X_train, X_val = X_use.iloc[train_index],  X_use.iloc[test_index]
            Y_train, Y_val = Y_use.iloc[train_index],  Y_use.iloc[test_index]

            cos = pd.DataFrame(data=metrics.pairwise.cosine_similarity(X=X_train.transpose(), Y=Y_train.transpose(), dense_output=True))
            dict_cos_rank_val[i] = abs(cos).rank(axis=0, ascending=False).astype(int)
            i = i +1

    # test
    if run_test == True:
        print("prepare test set.")
        cos = pd.DataFrame(data=metrics.pairwise.cosine_similarity(X=X_use.transpose(), Y=Y_use.transpose(), dense_output=True))
        dict_cos_rank_test = abs(cos).rank(axis=0, ascending=False).astype(int)
    
    ############### 2. cross-validaton model fit and evaluate with test
    for mfs in method_feature_select[:2]:
        for topn in my_list_topn:
            if (topn == X.shape[1] ) & (mfs =='random'): ### random feature skip all features
                continue
            for m in list_model:
                print(time.ctime(), mfs.ljust(8,'-'), str(topn).ljust(6,'-'), m.ljust(10,'-'), end=":  ")
#                 continue
                if run_cross_validation:
                    start =time.time()
                    print("training split.", end=" ")

                    i = 0
                    for train_index, test_index in kf.split(Y_use): # random trials to stablize errors
                        X_train, X_val = X_use.iloc[train_index],  X_use.iloc[test_index]
                        Y_train, Y_val = Y_use.iloc[train_index], Y_use.iloc[test_index]
                        
                        cos_top = (dict_cos_rank_val[i] <= topn)
                        cos_top.columns = Y_train.columns
                        with Pool(nCPU) as pool:      
                            y_pred_in_rows = pool.map(comp_y_pred, Y_train.columns)
                        
                        y_pred =  pd.DataFrame(data=y_pred_in_rows, index=Y_val.columns, columns= Y_val.index).transpose().round(3)
                        res[mfs][topn][m]["y_pred_val"][i]           = y_pred
                        res[mfs][topn][m]["df_val_mae" ].loc[:, i] = [ metrics.mean_absolute_error(Y_val[g], y_pred[g]) for g in Y_use.columns]
                        res[mfs][topn][m]["df_val_rmse"].loc[:, i] = [ np.sqrt(metrics.mean_squared_error(Y_val[g], y_pred[g])) for g in Y_use.columns]
                        res[mfs][topn][m]["df_val_r2"].loc[:, i]      = [ np.corrcoef(Y_val[g], y_pred[g])[1,0] for g in Y_use.columns]
                        i = i + 1;  print(i, end=" ");  # end for split i
                    # end:  N fold
                    res[mfs][topn]["time"].loc[m, "val"] =  int(time.time() - start)
                    
                if  run_test:
                    start =time.time()
                    print("test set.", end=" ")

                    X_train, X_val, Y_train, Y_val = deepcopy(X_use),  deepcopy(X_test), deepcopy(Y_use),  deepcopy(Y_test)

                    cos_top = (dict_cos_rank_test <= topn)
                    cos_top.columns = Y_train.columns
                    with Pool(nCPU) as pool:      
                        y_pred_in_rows = pool.map(comp_y_pred, Y_train.columns)

                    y_pred =  pd.DataFrame(data=y_pred_in_rows, index=Y_val.columns, columns=Y_val.index).transpose().round(3)

                    res[mfs][topn][m]["y_pred_test"] = y_pred
                    i = 0
                    res[mfs][topn][m]["df_test_mae"].loc[:, i] = [ metrics.mean_absolute_error(Y_val[g], y_pred[g]) for g in Y_use.columns]
                    res[mfs][topn][m]["df_test_rmse"].loc[:, i] = [ np.sqrt(metrics.mean_squared_error(Y_val[g], y_pred[g])) for g in Y_use.columns]
                    res[mfs][topn][m]["df_test_r2"].loc[:, i] = [ np.corrcoef(Y_val[g], y_pred[g])[1,0] for g in Y_use.columns]
                    res[mfs][topn]["time"].loc[m, "test"] =  int(time.time() - start)
                    
                print("done.")
                
    with open('/public/home/test1/mydata/proteome/data/res_' + mykey + time.strftime("%Y%m%d-%H%M")+'.pkl', 'wb') as f:   
        pickle.dump( [res],  f)

colon_86_2014_labelfree (86, 14959) (86, 2244) (68, 14959) (18, 14959) (68, 2244) (18, 2244)
[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 200, 1000, 5000, 14959]
prepare training set.
prepare test set.
Sat Oct  2 22:55:57 2021 random-- 5----- LR--------:  training split. 1 2 3 4 5 test set. done.
Sat Oct  2 22:56:42 2021 random-- 5----- Lasso-----:  training split. 1 2 

/public/home/test1/soft/anaconda3/envs/py3.7.3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/public/home/test1/soft/anaconda3/envs/py3.7.3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


3 4 5 test set. done.
Sat Oct  2 22:57:28 2021 random-- 5----- HR--------:  training split. 1 2 3 4 5 test set. done.
Sat Oct  2 22:58:16 2021 random-- 5----- Ridge-----:  training split. 1 2 3 4 5 test set. done.
Sat Oct  2 22:59:03 2021 random-- 5----- SVR-------:  training split. 1 2 3 4 5 test set. done.
Sat Oct  2 22:59:49 2021 random-- 5----- RFR-------:  training split. 1 2 3 4 5 test set. done.
Sat Oct  2 23:01:32 2021 random-- 10---- LR--------:  training split. 1 2 3 4 5 test set. done.
Sat Oct  2 23:02:18 2021 random-- 10---- Lasso-----:  training split. 1 2 3 4 5 test set. done.
Sat Oct  2 23:03:05 2021 random-- 10---- HR--------:  training split. 1 2 3 4 5 test set. done.
Sat Oct  2 23:03:54 2021 random-- 10---- Ridge-----:  training split. 1 2 3 4 5 test set. done.
Sat Oct  2 23:04:42 2021 random-- 10---- SVR-------:  training split. 1 2 3 4 5 test set. done.
Sat Oct  2 23:05:29 2021 random-- 10---- RFR-------:  training split. 1 2 3 4 5 test set. done.
Sat Oct  2 23:07:1

In [ ]:
#### random all == cosine all

In [15]:
res[mfs][topn]["time"]

,val,test
LR,193,45
Lasso,547,125
HR,1492,425
Ridge,251,39
SVR,299,79
RFR,12260,3220


In [ ]:
## summary for a fixed topN, average meric of all genes
mfs = "cosine"
print(mfs)
# my_list_topn = list_topn
for topN in my_list_topn:
    print(topN, end="_")
    res[mfs][topN]["summary"].index.name = mfs + '_' +str(topN)
    for m in model_dict:
        for my_metric in ["mae","rmse","r2"]:

            res[mfs][topN]["summary"].loc[m, my_metric + "_val_mean"] =  res[mfs][topN][m]["df_val_"+ my_metric].mean(axis=1).mean()

            res[mfs][topN]["summary"].loc[m, my_metric + "_val"] =  \
                "%.3f" % res[mfs][topN]["summary"].loc[m, my_metric + "_val_mean"] +  "±" + \
                "%.3f" % res[mfs][topN][m]["df_val_"+ my_metric].mean(axis=1).std()

            res[mfs][topN]["summary"].loc[m, my_metric + "_test_mean"] =  res[mfs][topN][m]["df_test_"+ my_metric].mean(axis=1).mean()

            res[mfs][topN]["summary"].loc[m, my_metric + "_test"] =  \
                "%.3f" % res[mfs][topN]["summary"].loc[m, my_metric + "_test_mean"] +  "±" + \
                "%.3f" % res[mfs][topN][m]["df_test_"+ my_metric].mean(axis=1).std()

res[mfs][topN]["summary"].iloc[:, 9:].astype(np.float).style.highlight_min(axis=0)
# 0.781±0.142	0.997±0.187	0.158±0.238

In [13]:
## summary for models at different feature length
# my_list_topn = list_topn
df_template = pd.DataFrame(index = list(model_dict.keys()),  columns = my_list_topn)
df_time, df_mae, df_rmse, df_r2= df_template.copy(),df_template.copy(), df_template.copy(), df_template.copy() 
mfs = "cosine"
# mfs = "random"
for topN in my_list_topn:
    print(topN, end="_")
    
    df_time.loc[:, topN] = res[mfs][topN]["time"].loc[:,"test"]
    df_mae.loc[:, topN] = res[mfs][topN]["summary"].loc[:, "mae_test_mean"]
    df_rmse.loc[:, topN] = res[mfs][topN]["summary"].loc[:, "rmse_test_mean"]
    df_r2.loc[:, topN] = res[mfs][topN]["summary"].loc[:, "r2_test_mean"]
    

5_10_15_20_25_30_35_40_45_50_200_1000_5000_10163_

In [14]:
df_r2.astype(np.float)#.style.highlight_max(axis=1)

,5,10,15,20,25,30,35,40,45,50,200,1000,5000,10163
LR,0.459370,0.461265,0.453990,0.442301,0.429772,0.414981,0.398039,0.382294,0.365553,0.348935,0.349120,0.464322,0.484082,0.484515
Lasso,0.462317,0.473300,0.477296,0.477022,0.476862,0.475652,0.473417,0.471808,0.469414,0.467784,0.448773,0.451986,0.467608,0.472986
HR,0.460767,0.462186,0.454125,0.441155,0.428510,0.412761,0.395502,0.377796,0.358206,0.338837,0.363951,0.462043,0.477458,0.476920
Ridge,0.461118,0.465800,0.461584,0.452875,0.444051,0.433192,0.420978,0.409683,0.397792,0.387499,0.362935,0.464643,0.484095,0.484571
SVR,0.425585,0.439023,0.446553,0.449984,0.453421,0.455775,0.457474,0.458747,0.460272,0.461518,0.467451,0.462425,0.458914,0.457214
RFR,0.450003,0.469952,0.477901,0.481831,0.484714,0.486606,0.487644,0.489023,0.489835,0.490968,0.495729,0.494604,0.490995,0.489060
